In [15]:
import numpy as np

from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import figure, show
from bokeh.io import output_file
from bokeh.resources import CDN
from bokeh.embed import file_html, json_item
import json
import IPython

# output_file("/home/peter/Documents/cs183/tmp/sinslider.html")


# DEFINE VARIABLES !!!

ELECTRON_CHARGE = 0.1
CG1 = 0.5
CG2 = 0.5
CL = 0.2
CR = 0.2
CM = 0.3 #1e-5
C1 = CL+CG1+CM
C2 = CR+CG2+CM

EC1 = ELECTRON_CHARGE**2/C1 * (1/(1-CM**2/(C1*C2)))
EC2 = ELECTRON_CHARGE**2/C2 * (1/(1-CM**2/(C1*C2)))
ECM = ELECTRON_CHARGE**2/CM * (1/(C1*C2/CM**2 - 1))

N=4

n_arr = np.arange(N)

v1_type1 = []
v2_type1 = []
v1_type2 = []
v2_type2 = []

for A in n_arr:
    for B in n_arr:
        # triple point type 1
        v11 = ELECTRON_CHARGE/CG1 * (A + EC2/2 * (EC1-ECM)/(EC1*EC2-ECM**2))
        v1_type1.append(v11)
        v2_type1.append((ELECTRON_CHARGE*((B+.5)*EC2 + A*ECM) - CG1*v11*ECM)/(CG2*EC2))
        # triple point type 2
        v12 = ELECTRON_CHARGE/CG1 * ((A+1) + EC2/2 * (EC1-ECM)/(ECM**2-EC1*EC2))
        v1_type2.append(v12)
        v2_type2.append((ELECTRON_CHARGE*((A+1)*ECM + ((B+1)-.5)*EC2) - CG1*v12*ECM)/(CG2*EC2))

source = ColumnDataSource(data=dict(v1_t1=v1_type1, v2_t1=v2_type1, v1_t2=v1_type2, v2_t2=v2_type2))

plot = figure(y_range=(0, 1), x_range=(0, 1), width=400, height=400)

plot.circle(x='v1_t1', y='v2_t1', source=source)
plot.circle(x='v1_t2', y='v2_t2', source=source)

cm_slider = Slider(start=0.001, end=10, value=1, step=0.1, title='C_M')

callback = CustomJS(args=dict(source=source, cm=cm_slider, ec=ELECTRON_CHARGE, cg1=CG1, cg2=CG2, cl=CL, cr=CR, n=N),
                    code="""
    const CM = cm.value
    const EC = ec

    const CG1 = cg1
    const CG2 = cg2
    const CL = cl
    const CR = cr
    const N = n
    const C1 = CL+CG1+CM
    const C2 = CR+CG2+CM

    const EC1 = EC*EC/C1 * (1/(1-CM*CM/(C1*C2)))
    const EC2 = EC*EC/C2 * (1/(1-CM*CM/(C1*C2)))
    const ECM = EC*EC/CM * (1/(C1*C2/CM*CM - 1))

    const v1_t1 = source.data.v1_t1
    const v2_t1 = source.data.v2_t1
    
    const v1_t2 = source.data.v1_t2;
    const v2_t2 = source.data.v2_t2;


    for (let a = 0; a < N; a++) {
        for (let b = 0; b < N; b++) {
            // triple point type 1
            const i = a * N + b
            v1_t1[i] = EC/CG1 * (a + EC2/2 * (EC1-ECM)/(EC1*EC2-ECM*ECM))
            v1_t2[i] = (EC*((b+.5)*EC2 + a*ECM) - CG1*v1_t1[i]*ECM)/(CG2*EC2)
            // v1_t1[i] = 0
            // v2_t1[i] = 0
        }
    }
    console.log("aye")
    console.log(v1_t1[0])
    console.log(CM)
    console.log(N)

    source.data = { v1_t1, v2_t1, v1_t2, v2_t2 }
""")

cm_slider.js_on_change('value', callback)

# item_text = json.dumps(json_item(row(plot, column(amp, freq, phase, offset)), "sinsliders"))
# item = json.parse(item_text)
# embed_item(item);
html_repr = file_html(row(plot, column(cm_slider)), CDN)
IPython.display.HTML(html_repr)
# show(row(plot, column(amp, freq, phase, offset)))
with open('./test.html', 'w+') as f:
    f.write(html_repr)